# Path4GMNS  

Note: This notebook is adapted from [tutorial.ipynb](https://github.com/jdlph/Path4GMNS/blob/master/tests/tutorial.ipynb) in the Path4GMNS Github repo, although it adds some notes and context after experimenting with the package and examining the source code.  

Path4GMNS is a Python package designed to work with networks in GMNS format.  

Path4GMNS supports:  
1. Static Traffic Assignment
2. User Equilibrium (UE) Traffic Assignment
3. Dynamic Traffic Assignment
4. Multimodal Accessibility and Equity Evaluation
5. Zone and Origin-Destination (OD) demand synthesis for a network

### 1. Install path4gmns

In [ ]:
pip install path4gmns

### 2. Import the module

In [ ]:
import path4gmns as pg

The minimum data required to work with Path4GMNS is a node.csv and link.csv file. We can prepare this data using the osm2gmns module

In [ ]:
import osm2gmns as og
net_id = 1159228

# download data and build network object
og.downloadOSMData(net_id)
net = og.getNetFromFile(POI=True)

# save network data in GMNS format
og.outputNetToCSV(net)

### 3. Read GMNS data  

Use `read_network()` to load the data from the link and node files into a Network object.  

You can specify the directory with the network data using the `input_dir` argument.  
You can also specify the units used in the network with the `length_unit` and `speed_unit` parameters. The supported units are as follows:    

Length units  
`'meter'` or `'m'`  
`'kilometer'` or `'km'`  
`'mile'` or `'mi'`  
  
Speed units  
`'kmh'` or `'kph'`  
`'mph'`  

The function will default to miles and miles per hour if no units are specified.

In [ ]:
net = pg.read_network(length_unit='meter')

### 4. Find the shortest path between 2 nodes

`network.find_shortest_path()` takes two node ids and returns a string detailing the total distance and the shortest path between the two, in a sequence of node ids.  

You can set the `seq_type` parameter to `'link'` to have the path described in a sequence of links

In [ ]:
path = net.find_shortest_path(from_node_id=1, to_node_id=2)
print(path)

path = net.find_shortest_path(1, 2, seq_type='link')
print(path)

### 5. Load Zones and OD Demand Matrix  



Zone information is necessary for conducting traffic assignment, evaluating accessibility and equity.  

If you don't have this data, you can synthesize it, as well as Origin-Destination demand matrix.  

Note: synthesizing zone and demand data will overwrite any existing `zone.csv` and `demand.csv` files you may already have.  
Note: to synthesize demand data, you will need to have a `poi.csv` file present in the directory where you called `read_network()`

In [ ]:
pg.network_to_zones(net)
pg.output_zones(net)

pg.output_synthesized_demand(net)

If you already have `zone.csv` and `demand.csv`, you can just load the zones and OD demand matrix with the following functions:

In [ ]:
net = pg.read_network()

pg.read_zones(net)
pg.load_demand(net)

Note: `read_zones()` and `load_demand()` will read `zones.csv` and `demand.csv` by default, respectively. You can specify different .csv files using the `filename` parameter in both functions.

### 6. Path-Based User Rquilibrium (UE) Traffic Assignment

OD demand matrix is necessary for this functionality, so you'll need to load the demand data before you can conduct the following traffic assignment 

In [ ]:
net = pg.read_network()

pg.read_zones(net)
pg.load_demand(net)

# specify parameters for traffic assignment 
column_gen_num = 10
column_update_num = 10

# path-based UE only
pg.perform_column_generation(column_gen_num, column_update_num, net)

# set the parameter output_geometry to False
# if you don't want to include geometry info into the output file
# output column information to agent.csv
pg.output_columns(net)

# output link performance information to link_performance.csv
pg.output_link_performance(net)

### 7. Evaluate Accessibility

Accessibility defines where you can go given a time budget and a transportation mode (e.g., auto).  
You can find the number of accessible zones from each zone (`zone_accessibility.csv`) along with the free flow travel time for each OD pair in minutes (`od_accessibility.csv`)

Note: the default mode is `'auto'` and the default time budget is 240 minutes.
Note: Zone information is necessary for accessibility evaluation
Note: Make sure to set `single_mode` to True for single mode evaluation. Multimodal evaluation will be visited in a later section.

In [ ]:
# No need to load demand information for accessibility evaluation
net = pg.read_network()
pg.read_zones(net)

pg.evaluate_accessibility(net, single_mode=True)

### 8. Evaluate Equity

Transportation Equity is accessibility with respect to different demographics. Given a time budget and a segmentation of zones, Path4GMNS provides the following simple info and statistics on equity:

1. accessible zones
1. min accessibility (The zone with the minimum number of accessible zones)
1. max accessibility (The zone with the maximum number of accessible zones)
1. mean accessibility (The average number of accessible zones over a bin of zones)

In [ ]:
# No need to laod demand information for equity evaluation
net = pg.read_network()
pg.read_zones(net)

# single_mode must be set to True for single mode evaluation
pg.evaluate_equity(net, single_mode=True)

### 9. Multimodal Evaluation

For multimodal evaluation, the corresponding modes (i.e., agent types) must be present in `settings.yml`. For this, we will use the pyyaml Python package (version 5.1 or higher)

In [ ]:
pip install pyyaml

If you don't hae a YAML settings file, you can download a base `settings.yml` file using the following function:

In [ ]:
# pg.download_sample_setting_file()

Note: working with the sample setting file may return the error:  
`Invalid time_periods:  not enough values to unpack (expected 2, got 1)`  

If this happens, try changing the following line in `settings.yml`:  
`    time_period: 0700-0800`  

to:  
`    time_period: 0700_0800`

### Get the Shortest Path Between Two Nodes Under a Specific Mode

Use the function `find_shortest_path()` specifying the mode you want to evaluate in the `mode` parameter. 
 
Note: The mode you want to use has to be defined in `settings.yml`  
Note: `find_shortest_path()` will not return a valid path if there is not a possible path where the links support the mode specified in the function. You can check the supported modes of each link in the `allowed_uses` attribute of link.csv

In [ ]:
net = pg.read_network(length_unit='meter')

# mode 'a' for auto. You can also use mode 'w' for walk
path = net.find_shortest_path(1, 2, mode='a')
print(path)

path = net.find_shortest_path(1, 2, mode='a', seq_type='link')
print(path)

### Multimodal Accessibility Evaluation

Multimodal accessibility evaluation is similar to single mode accessibility evaluation. You just need to set the `single_mode` parameter in `evaluate_accessibility()` to `False`

In [ ]:
net = pg.read_network()
pg.read_zones(net)

# single_mode parameter is set to False by default
pg.evaluate_accessibility(net)

### Multimodal Equity Evaluation

Just like with multimodal accessibility evaluation, for multimodal equity evaluation you just need to set `single_mode` to `False` in `evaluate_equity()`

In [ ]:
net = pg.read_network()
pg.read_zones(net)

# single_mode parameter is set to False by default
pg.evaluate_equity(net)

### Conduct Dynamic Traffic Simulation

Traffic simulation requires routing decisions from each agent, which is obtained from User Equilibrium traffic assingment.

In [ ]:
net = pg.read_network()

pg.read_zones(net)
pg.load_demand(net)

# User Equilibrium + Dynamic Traffic Assingment
column_gen_num = 10
column_update_num = 10

pg.perform_column_generation(column_gen_num, column_update_num, net)
pg.perform_simple_simulation(net)
print('Completed dynamic simulation\n')

print('writing agent trajectories')
pg.output_agent_trajectory(net)

If you already have an agent.csv file, you can just load it and skip the column generation step

In [ ]:
net = pg.read_network()

# Load prepared User Equilibrium data
pg.load_columns(net)

# Perform Dynamic Traffic Assignemnt
pg.perform_simple_simulation(net)
print('Completed dynamic simulation\n')

print('writing agent trajectories')
pg.output_agent_trajectory(net)

To obtain the shortest path, you can set `column_gen_num` to 1 and `column_update_num` to 0

In [ ]:
net = pg.read_network()

pg.read_zones(net)
pg.load_demand(net)

# User Equilibrium + Dynamic Traffic Assingment
column_gen_num = 1
column_update_num = 0

pg.perform_column_generation(column_gen_num, column_update_num, net)
pg.perform_simple_simulation(net)
print('Completed dynamic simulation\n')

print('writing agent trajectories')
pg.output_agent_trajectory(net)

### In Case of Special Events

A special event may come with capacity reduction over the affected links. You can introduce one special event for each demand period in `settings.yml` as below:

```yaml
demand_periods:
  - period: AM
    time_period: 0700_0800
    special_event:
      name: work_zone
      enable: true
      # with respect to iterations in column generation
      beg_iteration: 1
      end_iteration: 20
      affected_links:
        - link_id: 1
          capacity_ratio: 0.5
        - link_id: 2
          capacity_ratio: 0.4
        - link_id: 3
          capacity_ratio: 0.6
        - link_id: 4
          capacity_ratio: 0
```

If the original capacity of an affected link is C, its affected capacity will be r * C, where r is the reduction ratio (`capacity_ratio`) when a special event is present. setting the `capacity_ratio` of an affected link to 0 is equivalent to removing the link from the entire demand period. You can turn on or off a special event by setting `special_event.enable` to true or false in the settings file.

TODO: conduct traffic assignment with the special events

### Accessibility Considering Time-Dependent Link Travel Time

For single mode and multimodal accessibility evaluation, accessibility is evaluated using the link free flow travel time (FFTT), which is determined by link length and link free flow speed under a specific mode.  

Link travel time varies over time, and so does accessibility. When the time-dependent accessibility is of interest, time-dependent link travel time (i.e., VDF_fftt from a given demand period in `link.csv`) will come into play by overwriting the static link free flow speed (from either `link.csv` or `settings.yml`. Both are denoted as `free_speed`).

In [ ]:
net = pg.read_network()

# time-dependent accessibility under the default mode auto
# for demand period 0
pg.evaluate_accessibility(net, single_mode=True, time_dependent=True)

# time-dependent accessibility under a specific mode, walk
pg.evaluate_accessibility(net, single_mode=True, mode='w', time_dependent=True)

### DTALite

Path4GMNS also serves as an API to the C++ based DTALite to conduct various multimodal traffic assignments, including:  

```
0: Link-based User Equilibrium (UE)
1: Path-based User Equilibrium (UE)
2: UE + Dynamic Traffic Assignment (DTA)
3: OD Matrix Estimation (ODME)
```

Note: Make sure to restart the kernel after every `perform_network_assignment_DTALite()` run.

### Link-Based User Equilibrium

In [ ]:
# No need to call read_network() as network and demand will be loaded by DTALite

mode = 0
column_gen_num = 10
column_update_num = 10

pg.perform_network_assignment_DTALite(mode, column_gen_num, column_update_num)

### Path-Based User Equilibrium

This mode of traffic assignment requires a `settings.csv` file

In [ ]:
import path4gmns as pg

mode = 1
column_gen_num = 10
column_update_num = 10

pg.perform_network_assignment_DTALite(mode, column_gen_num, column_update_num)

### User Equilibrium + Dynamic Traffic Assignment

In [ ]:
import path4gmns as pg

mode = 2
column_gen_num = 10
column_update_num = 10

pg.perform_network_assignment_DTALite(mode, column_gen_num, column_update_num)

### OD Matrix Estimation

In [ ]:
import path4gmns as pg

mode = 3
column_gen_num = 10
column_update_num = 10

pg.perform_network_assignment_DTALite(mode, column_gen_num, column_update_num)